<img src="https://www.escoladnc.com.br/wp-content/uploads/2022/06/dnc_formacao_dados_logo_principal_preto-1.svg" alt="drawing" width="300"/>

# Recomendação Top N

Este notebook contém 2 exemplos de recomendação Top N:

- **Top N consumidos**: os N itens mais consumidos pelos usuários
- **Top N avaliados**: os N itens melhores avaliados pelos usuários

O dataset a ser utilizado será o [MovieLens](https://grouplens.org/datasets/movielens/), cuja análise exploratória foi feita no exemplo prático do módulo **Introdução aos Sistemas de Recomendação**. 

In [6]:
import os
import sys
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#007efd', '#FFC000', '#303030'])

# Carregando e processando o dataset

Para mais informações desta sessão, consulte o notebook `Análise Exploratória do MovieLens` do módulo 01.

## Arquivo de avaliações

Upload file `ratings.parquet`

In [2]:
%%time
_ = files.upload() # approx: 1min56s

Saving ratings.parquet to ratings.parquet
CPU times: user 1.4 s, sys: 171 ms, total: 1.57 s
Wall time: 1min 24s


In [7]:
df_ratings = pd.read_parquet('ratings.parquet')
df_ratings.tail()

,user_id,item_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


## Arquivo de metadados dos itens

Upload file `movies.parquet`

In [3]:
%%time
_ = files.upload() # approx: 10s

Saving movies.parquet to movies.parquet
CPU times: user 157 ms, sys: 17.3 ms, total: 174 ms
Wall time: 10 s


In [5]:
def convert_genres_to_list(genres:str, separator='|'):
    return genres.split(separator)

df_items = pd.read_parquet('movies.parquet')
df_items['genres'] = df_items['genres'].apply(convert_genres_to_list)
df_items.tail()

,item_id,title,genres
3878,3948,Meet the Parents (2000),[Comedy]
3879,3949,Requiem for a Dream (2000),[Drama]
3880,3950,Tigerland (2000),[Drama]
3881,3951,Two Family House (2000),[Drama]
3882,3952,"Contender, The (2000)","[Drama, Thriller]"


# Recomendação Top N Consumidos

Em nosso primeiro exemplo de recomendação não-personalizada iremos recomendar os items mais consumidos pelos usuários. 

Em geral, uma função de recomendação retorna 2 tipos de informação:

- `item_id`: identificador do item
- `score`: _score_ a ser utilizado para ordenação da oferta para o usuário

In [13]:
def recommend_top_n_consumptions(ratings:pd.DataFrame, n:int) -> pd.DataFrame:

    recommendations = (
        ratings
        .groupby('item_id')
        .count()['user_id']
        .reset_index()
        .rename({'user_id': 'score'}, axis=1)
        .sort_values(by='score', ascending=False)
    )

    return recommendations.head(n)

df_top_consumptions = recommend_top_n_consumptions(df_ratings, n=10)
df_top_consumptions

,item_id,score
2651,2858,3428
253,260,2991
1106,1196,2990
1120,1210,2883
466,480,2672
1848,2028,2653
575,589,2649
2374,2571,2590
1178,1270,2583
579,593,2578


Para melhor avaliar o resultado da recomendação, podemos **anexar os metadados dos itens**

In [ ]:
df_top_consumptions.merge(df_items, on='item_id', how='inner')

,item_id,score,title,genres
0,2858,3428,American Beauty (1999),"[Comedy, Drama]"
1,260,2991,Star Wars: Episode IV - A New Hope (1977),"[Action, Adventure, Fantasy, Sci-Fi]"
2,1196,2990,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Drama, Sci-Fi, War]"
3,1210,2883,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Romance, Sci-Fi, War]"
4,480,2672,Jurassic Park (1993),"[Action, Adventure, Sci-Fi]"
5,2028,2653,Saving Private Ryan (1998),"[Action, Drama, War]"
6,589,2649,Terminator 2: Judgment Day (1991),"[Action, Sci-Fi, Thriller]"
7,2571,2590,"Matrix, The (1999)","[Action, Sci-Fi, Thriller]"
8,1270,2583,Back to the Future (1985),"[Comedy, Sci-Fi]"
9,593,2578,"Silence of the Lambs, The (1991)","[Drama, Thriller]"


____________

# Recomendação Top N Avaliados

Uma outra abordagem de recomendação top-N é considerar os itens melhores avaliados pelos usuários utilizando os campos de feedback explícito, como o _rating_. Para isso, utilizaremos a **avaliação média de um filme** no dataset do MovieLens.

In [24]:
def recommend_top_n_evaluations(ratings:pd.DataFrame, n:int, min_evaluations:int=None) -> pd.DataFrame:
    recommendations = (
        ratings
        .groupby('item_id')
        .agg({'rating': 'mean', 'user_id': 'count'})
        .reset_index()
        .rename({'rating': 'score', 'user_id': 'evaluations'}, axis=1)
        .sort_values(by=['score', 'evaluations'], ascending=False)
    )

    if min_evaluations is not None:
        recommendations = recommendations.query('evaluations >= @min_evaluations')

    return recommendations.head(n)

recommend_top_n_evaluations(df_ratings, n=10, min_evaluations=None)

,item_id,score,evaluations
744,787,5.0,3
3010,3233,5.0,2
926,989,5.0,1
1652,1830,5.0,1
2955,3172,5.0,1
3054,3280,5.0,1
3152,3382,5.0,1
3367,3607,5.0,1
3414,3656,5.0,1
3635,3881,5.0,1


In [25]:
df_top_evaluations = recommend_top_n_evaluations(df_ratings, n=10, min_evaluations=None)
df_top_evaluations.merge(df_items, on='item_id', how='inner')

,item_id,score,evaluations,title,genres
0,787,5.0,3,"Gate of Heavenly Peace, The (1995)",[Documentary]
1,3233,5.0,2,Smashing Time (1967),[Comedy]
2,989,5.0,1,Schlafes Bruder (Brother of Sleep) (1995),[Drama]
3,1830,5.0,1,Follow the Bitch (1998),[Comedy]
4,3172,5.0,1,Ulysses (Ulisse) (1954),[Adventure]
5,3280,5.0,1,"Baby, The (1973)",[Horror]
6,3382,5.0,1,Song of Freedom (1936),[Drama]
7,3607,5.0,1,One Little Indian (1973),"[Comedy, Drama, Western]"
8,3656,5.0,1,Lured (1947),[Crime]
9,3881,5.0,1,Bittersweet Motel (2000),[Documentary]


Note que alguns itens podem ter avaliações altas, porém elas podem ter sido dadas por poucos usuários. Assim, podemos incluir um **hiperparâmetro** com a **quantidade mínima de avaliações** que um item precisa ter para ser considerado na recomendação.

In [26]:
df_top_evaluations = recommend_top_n_evaluations(df_ratings, n=10, min_evaluations=100)
df_top_evaluations.merge(df_items, on='item_id', how='inner')

,item_id,score,evaluations,title,genres
0,2019,4.560510,628,Seven Samurai (The Magnificent Seven) (Shichin...,"[Action, Drama]"
1,318,4.554558,2227,"Shawshank Redemption, The (1994)",[Drama]
2,858,4.524966,2223,"Godfather, The (1972)","[Action, Crime, Drama]"
3,745,4.520548,657,"Close Shave, A (1995)","[Animation, Comedy, Thriller]"
4,50,4.517106,1783,"Usual Suspects, The (1995)","[Crime, Thriller]"
5,527,4.510417,2304,Schindler's List (1993),"[Drama, War]"
6,1148,4.507937,882,"Wrong Trousers, The (1993)","[Animation, Comedy]"
7,922,4.491489,470,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),[Film-Noir]
8,1198,4.477725,2514,Raiders of the Lost Ark (1981),"[Action, Adventure]"
9,904,4.476190,1050,Rear Window (1954),"[Mystery, Thriller]"


Assim como em modelos de _Machine Learning_ ou _Deep Learning_, os hiperparâmetros de uma recomendação não-personalizada precisam ser validados com experimentos _offline_ ou _online_. O tópico de experimentação será abordado mais à frente neste curso.

_____________

**Extra**: geração de resultados para avaliação das métricas no módulo 05

In [ ]:
import numpy as np
train_size = 0.8
df_ratings.sort_values(by='timestamp', inplace=True)
df_train_set, df_valid_set= np.split(df_ratings, [int(train_size * df_ratings.shape[0])])

In [ ]:
recommendations = recommend_top_n_consumptions(df_ratings, n=20)
scores = [{'item_id': x['item_id'], 'score': x['score']} for _, x in recommendations.iterrows()]

In [ ]:
from tqdm import tqdm
model_name = 'top'
df_predictions = df_valid_set
df_predictions['y_true'] = df_predictions.apply(lambda x: {'item_id': x['item_id'], 'rating': x['rating']}, axis=1)
df_predictions = df_predictions.groupby('user_id').agg({'y_true': list})
df_predictions['y_score'] = df_predictions.apply(lambda x: scores, axis=1)
df_predictions['model'] = model_name
df_predictions.reset_index(drop=False, inplace=True)
df_predictions.tail()


,user_id,y_true,y_score,model
1778,6001,"[{'item_id': 3751, 'rating': 4}, {'item_id': 3...","[{'item_id': 2858, 'score': 3428}, {'item_id':...",top
1779,6002,"[{'item_id': 1942, 'rating': 5}, {'item_id': 4...","[{'item_id': 2858, 'score': 3428}, {'item_id':...",top
1780,6016,"[{'item_id': 3756, 'rating': 3}, {'item_id': 3...","[{'item_id': 2858, 'score': 3428}, {'item_id':...",top
1781,6028,"[{'item_id': 3000, 'rating': 4}]","[{'item_id': 2858, 'score': 3428}, {'item_id':...",top
1782,6040,"[{'item_id': 3182, 'rating': 5}, {'item_id': 2...","[{'item_id': 2858, 'score': 3428}, {'item_id':...",top


In [ ]:
column_order = ['model', 'user_id', 'y_true', 'y_score']
df_predictions[column_order].to_parquet(f'valid_{model_name}.parquet', index=None)